# Linear Algebra

## Dense / full eigen decomposition

Currently full decompositions use numpy. They are as follows:

> - {func}`~quimb.linalg.base_linalg.eig`
> - {func}`~quimb.linalg.base_linalg.eigh`
> - {func}`~quimb.linalg.base_linalg.eigvals`
> - {func}`~quimb.linalg.base_linalg.eigvalsh`
> - {func}`~quimb.linalg.base_linalg.eigvecs`
> - {func}`~quimb.linalg.base_linalg.eigvecsh`
> - {func}`~quimb.linalg.base_linalg.eigensystem`
> - {func}`~quimb.linalg.base_linalg.svd`

### Automatic Exploitation of Symmetries via Blocking

Sometimes symmetries present themselves in an operator as, after
a certain permutation, diagonal blocks. This fact can be exploited
to speed up full eigen-decompositions as each block can be
decomposed individually. For example, imagine we want to supply
the eigen-decomposition of the Heisenberg Hamiltonian to an
{class}`~quimb.evo.Evolution` object to perform long-time, exact,
dynamics:


In [1]:
import quimb as qu

H = qu.ham_heis(12)

In [2]:
%%timeit
el, ev = qu.eigh(H)

9.17 s ± 850 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This is already quite slow, and will not be sustainable for larger lengths. 
The Heisenberg Hamiltonian however has an abelian $Z_2$ subgroup symmetry, 
conserved Z-spin, that manifests itself in the computational basis.
Thus if instead we specify ``autoblock=True``:

In [3]:
%%timeit
el, ev = qu.eigh(H, autoblock=True)

466 ms ± 63.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can see a drastic speedup (helped by the fact that the whole algorithm uses `numba`).
To work more fundamentally with the indentified blocks see the
{func}`~quimb.linalg.autoblock.compute_blocks` function.

## Partial eigen decomposition

Partial decompositions are mostly just specified by  supplying the `k` kwarg to the above functions. These also take a `backend` argument which can be one of:

- `'scipy'`: is generally reliable
- `'numpy'`: can be faster for small or dense problems
- `'lobpcg'`: useful for fast, low accruacy generalized eigenproblems (like periodic DMRG)
- `'slepc'`: Usually the fastest for large problems, with many options. Will either spawn MPI
  workers or should be used in `syncro` mode.
- `'slepc-nompi'`: like `'slepc'`, but performs computation in the single, main process.
- `'AUTO'` - choose a good backend, the default.

The possible partical decompositions are:

> - {func}`~quimb.linalg.base_linalg.eig` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigh` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigvals` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigvalsh` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigenvectors` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigvecs` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.eigvecsh` with `(k > 0)`
> - {func}`~quimb.linalg.base_linalg.groundstate`
> - {func}`~quimb.linalg.base_linalg.groundenergy`
> - {func}`~quimb.linalg.base_linalg.eigensystem_partial`
> - {func}`~quimb.linalg.base_linalg.svds`

So for example the {func}`~quimb.linalg.base_linalg.groundstate` function
for a Hamiltonian `H` is an alias to:

In [5]:
psi = qu.eigvecsh(H, k=1, which='SA')

\[find eigenvectors, Hermitian operator (`h` post-fix), get `k=1` eigenstate,
and target the '(s)mallest (a)lgebraic' eigenvalue\].

## Interior eigen-solving

SLEPc is highly recommended for performing these using 'shift-invert'.
See the following functions:

> - `eigh(..., k=k, sigma=x)` with `k > 0` etc., or
> - {func}`~quimb.linalg.base_linalg.eigh_window`
> - {func}`~quimb.linalg.base_linalg.eigvalsh_window`
> - {func}`~quimb.linalg.base_linalg.eigvecsh_window`

With the last three allowing the specification of a window *relative* to the total spectrum of the operator.

## Fast Randomized Linear Algebra

`quimb` has an implementation of a fast randomized SVD - {func}`~quimb.linalg.rand_linalg.rsvd` -
that can be significantly quicker than {func}`~quimb.linalg.base_linalg.svd` or {func}`~quimb.linalg.base_linalg.svds`,
especially for large `k`. This might be useful for e.g. tensor network linear operator decompositions.
It can perform the SVD rank-adaptively, which allows the efficient estimation of an operator's rank,
see {func}`~quimb.linalg.rand_linalg.estimate_rank`.